In [58]:
import numpy as np
from scipy.linalg import expm
from qutip import *
from qsq_protocol import *

In [59]:
# Function 1: Calculate U from theta1, theta2, theta3


# Example usage:
theta1, theta2, theta3 = np.pi/2, np.pi/3, np.pi/6
print(theta1,theta2,theta3)
U = compute_U(theta1, theta2, theta3)
print("U:", U)

# Decompose U back into theta1, theta2, theta3
theta1_reconstructed, theta2_reconstructed, theta3_reconstructed = decompose_U(U)
print(f"Reconstructed theta1: {theta1_reconstructed}, theta2: {theta2_reconstructed}, theta3: {theta3_reconstructed}")


1.5707963267948966 1.0471975511965976 0.5235987755982988
U: Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[1.76762540e-17+0.28867513j 4.33012702e-01+0.25j      ]
 [4.33012702e-01-0.25j       1.76762540e-17-0.28867513j]]
Reconstructed theta1: 1.5707963267948966, theta2: 0.5235987755982988, theta3: 0.5235987755982988


In [60]:
rho_exp = rand_dm(2)
M_exp = ket2dm(rand_ket(2))
gate_exp = rand_unitary(2)
average_fidelity(rho_exp,gate_exp,M_exp)

np.float64(0.495028)

In [61]:
import numpy as np
from scipy.optimize import minimize, dual_annealing, basinhopping, differential_evolution
from qutip import rand_dm, rand_unitary

# Assuming you have compute_U, average_fidelity, decompose_U, plus_state, and minus_state defined

# Define the objective function to minimize
def objective_function(params, rho_exp, gate_exp, M_exp):
    # Unpack the parameters (theta1, theta2, theta3)
    theta1, theta2, theta3 = params
    
    # Compute the unitary operator U based on theta1, theta2, theta3
    U = compute_U(theta1, theta2, theta3)
    
    # Compute the objective function (average fidelity)
    return 1 - average_fidelity(U @ rho_exp @ U.dag(), U @ gate_exp @ U.dag(), U @ M_exp @ U.dag())

# Example fixed matrices (rho_exp, gate_exp, M_exp)
rho_exp = rand_dm(2).unit()
M_exp = rand_dm(2).unit()
gate_exp = rand_unitary(2).unit()
print("Initial fidelity:", average_fidelity(rho_exp, gate_exp, M_exp))

# Initial guess for the parameters (theta1, theta2, theta3)
eigenvalues, eigenstates = M_exp.eigenstates()
U_guess = eigenstates[0] @ plus_state.dag() - 1j * eigenstates[1] @ minus_state.dag()
initial_guess = decompose_U(U_guess)
print("Initial guess:", initial_guess)

# Define optimization function
def optimize_method(method, objective_function, initial_guess, rho_exp, gate_exp, M_exp, bounds=None):
    if method == 'BFGS' or method == 'COBYLA':
        result = minimize(
            objective_function, 
            initial_guess, 
            bounds=bounds,  
            args=(rho_exp, gate_exp, M_exp),  
            method=method
        )
    elif method == 'differential_evolution':
        result = differential_evolution(
            objective_function, 
            bounds=bounds,
            args=(rho_exp, gate_exp, M_exp),
            maxiter=3000,
            popsize=30
        )
    elif method == 'basinhopping':
        result = basinhopping(
            objective_function, 
            initial_guess,  
            minimizer_kwargs={"method": "COBYLA", "args": (rho_exp, gate_exp, M_exp)},  
        )
    else:
        raise ValueError("Unknown optimization method.")
    
    return result

# Define bounds for optimization
bounds = [(-np.pi, np.pi), (0, np.pi/2), (-np.pi, np.pi)]

# Test different optimization methods
methods = ['BFGS', 'COBYLA', 'differential_evolution', 'basinhopping']

for method in methods:
    print(f"\nOptimizing with {method}...")
    result = optimize_method(method, objective_function, initial_guess, rho_exp, gate_exp, M_exp, bounds)
    
    # Display the result
    if result.success:
        print(f"Optimal parameters: {result.x}")
        print(f"Objective function value at optimal parameters: {1 - result.fun}")
    else:
        print(f"Optimization failed: {result.message}")


Initial fidelity: 0.513347
Initial guess: (np.float64(-2.585523545490964), np.float64(0.7853981633974482), np.float64(2.585523545490964))

Optimizing with BFGS...
Optimal parameters: [-2.58552355  0.78539816  2.58552355]
Objective function value at optimal parameters: 0.571969

Optimizing with COBYLA...
Optimal parameters: [-2.58552355  0.78539816  2.58552355]
Objective function value at optimal parameters: 0.571969

Optimizing with differential_evolution...


C:\Users\XPS13-9360\AppData\Local\Temp\ipykernel_7124\3297133067.py:33: RuntimeWarning: Method BFGS cannot handle bounds.
  result = minimize(


Optimal parameters: [-1.78151771  0.78550132  1.83690011]
Objective function value at optimal parameters: 0.639902

Optimizing with basinhopping...
Optimal parameters: [-1.85622938  0.78545053  1.83293428]
Objective function value at optimal parameters: 0.639738


In [62]:
rho_exp = rand_dm(2).unit()
M_exp = rand_dm(2).unit()
gate_exp = rand_unitary(2).unit()
print(average_fidelity(rho_exp,gate_exp,M_exp))

0.558631


In [63]:
print(average_fidelity_gauge(rho_exp, gate_exp, M_exp))

0.6814
